# Протоколы транспортировки ключей

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

----

In [3]:
# КГПСЧ
from Crypto.Random import get_random_bytes

# Шифрование 
from Crypto.Cipher import AES

# Идентификаторы клиентов
import uuid

# Утилитки
from Crypto.Util import number
import binascii

-----

In [4]:
def bytes_as_hex(b: bytes) -> str:
    """
    Перевод бинарных данных в hex-строчку
    """
    
    return binascii.hexlify(b).decode()

----

## Протокол Нидхема-Шрёдера
![Needham-Schroeder Protocol](./images/Needham-Schroeder_3.png)

In [5]:
class NeedhamSchroederParams(object):
    SESSION_KEY_SIZE    = AES.key_size[2]
    ENCRYPTION_KEY_SIZE = AES.key_size[2]
    NUMBER_SIZE         = 4

In [6]:
class NeedhamSchroederCA(object):
    def __init__(self):
        """
        Инициализация доверенного центра.
        Создание пустой базы.
        """
        
        self._db = {}
        
    
    def register_client(self, client_id, client_key):
        """
        Регистрация клиента по его идентификатору и ключу.
        Если клиент уже есть, то бросает исключение.
        """
        
        trace('[NeedhamSchroederCA]', f'''Attempting to register client {client_id} 
        with key {bytes_as_hex(client_key)}''')
        
        if client_id in self._db:
            trace('[NeedhamSchroederCA]', f'Client {client_id} already exists')
            raise ValueError(f'Client {client_id} already exists')
            
        self._db[client_id] = client_key
        trace('[NeedhamSchroederCA]', 'Client registered successfully')
        
        
    def accept_initiation(self, first_id: uuid.UUID, second_id: uuid.UUID, number: bytes) -> bytes:
        """
        Выполнение функции ДЦ в процессе транспортировки ключа.
        """
        
        trace('[NeedhamSchroederCA]', f'Accepted initiation from {first_id} to {second_id}, Na = {bytes_as_hex(number)}')
        
        if first_id not in self._db or second_id not in self._db:
            trace('[NeedhamSchroederCA]', f"Client doesn't exist")
            raise ValueError(f"Client doesn't exist")
            
        #
        # Случайный ключ создаю
        #
        
        session_key = get_random_bytes(NeedhamSchroederParams.SESSION_KEY_SIZE)
        
        #
        # Зашифровываю внутренний пакет
        #
        
        cipher = AES.new(self._db[second_id], AES.MODE_CTR)
        nonce, internal_packet = cipher.nonce, cipher.encrypt(session_key + first_id.bytes)
        
        #
        # А теперь зашифровываю весь пакет
        #
        
        cipher = AES.new(self._db[first_id], AES.MODE_CTR)
        return cipher.nonce, cipher.encrypt(number + second_id.bytes + session_key + nonce + internal_packet)

In [7]:
class NeedhamSchroederClient(object):
    def __init__(self, fake_data: uuid.UUID = None):
        """
        Инициализация клиента с созданием идентификатора и генерацией ключа.
        Создается пустая база для сессионных ключей и челленджей.
        Второй параметр требуется для демонстрации неуспешной попытки исполнения протокола.
        """
        
        self._id  = uuid.uuid4() if fake_data is None else fake_data
        self._key = get_random_bytes(NeedhamSchroederParams.ENCRYPTION_KEY_SIZE)
        
        self._db  = {}
        self._challenges = {}
        
        trace('[NeedhamSchroederClient]', f'Client {self._id} created')
        
        
    def register(self, ca: NeedhamSchroederCA):
        """
        Регистрация на ДЦ.
        """
        
        ca.register_client(self._id, self._key)
        
        
    def initiate_protocol(self, second_client, ca: NeedhamSchroederCA):
        """
        Инициация протокола транспортировки ключа. Принимает дескрипторы
        второго клиента и доверенного центра.
        Возвращает статус успешности выполнения протокола.
        """
        
        N = get_random_bytes(NeedhamSchroederParams.NUMBER_SIZE)
        trace('[NeedhamSchroederClient]', f'[{self._id}] N = {bytes_as_hex(N)}')
        
        nonce, encrypted_data = ca.accept_initiation(self._id, second_client.identifier, N)
        
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=nonce)
        packet = cipher.decrypt(encrypted_data)
        
        if packet.find(N) != 0:
            trace('[NeedhamSchroederClient]', f'[{self._id}] Malformed response')
            return False
            
        #
        # Отрезаю лишнее и отправляю второму участнику нужные данные
        #
        
        packet = packet[len(N) + len(second_client.identifier.bytes):]
        session_key = packet[:NeedhamSchroederParams.SESSION_KEY_SIZE]
        
        #
        # Вычленяю nonce и шифртекст
        # Про размер nonce: If not present, the library creates a random nonce of length equal to block size/2.
        #
        
        packet = packet[NeedhamSchroederParams.SESSION_KEY_SIZE:]
        nonce, internal_packet = packet[:AES.block_size // 2], packet[AES.block_size // 2:]
        
        #
        # Получаю челлендж, модифицирую его и отправляю на проверку
        #
        
        challenge = second_client._accept_packet(nonce, internal_packet, self._id)
        if challenge is None:
            trace('[NeedhamSchroederClient]', f'[{self._id}] Second client denied packet')
            return False
        
        nonce, challenge = challenge
        cipher = AES.new(session_key, AES.MODE_CTR, nonce=nonce)
        
        challenge = cipher.decrypt(challenge)
        response = NeedhamSchroederClient._modify_challenge(challenge)
        
        cipher = AES.new(session_key, AES.MODE_CTR)
        if not second_client._verify_response(cipher.nonce, cipher.encrypt(response), self._id):
            trace('[NeedhamSchroederClient]', f'[{self._id}] Second client denied response')
            return False
        
        #
        # Проверки прошли - ключ общий есть
        #
        
        self._db[second_client.identifier] = session_key
        trace('[NeedhamSchroederClient]', f'[{self._id}] Established key: {bytes_as_hex(session_key)}')
        
        return True
    
    
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора клиента.
        """
        
        return self._id
        
        
    def _accept_packet(self, nonce: bytes, internal_packet: bytes, first_id: uuid.UUID):
        cipher = AES.new(self._key, AES.MODE_CTR, nonce=nonce)
        packet = cipher.decrypt(internal_packet)
        
        session_key = packet[:NeedhamSchroederParams.SESSION_KEY_SIZE]
        identifier  = packet[NeedhamSchroederParams.SESSION_KEY_SIZE:]
        
        #
        # Сравниваю идентификаторы
        #
        
        if identifier != first_id.bytes:
            trace('[NeedhamSchroederClient]', f'[{self._id}] Wrong identifier')
            return None
        
        #
        # Все ок, формирую челлендж
        #
        
        self._db[first_id] = session_key
        self._challenges[first_id] = get_random_bytes(NeedhamSchroederParams.NUMBER_SIZE)
        
        trace('[NeedhamSchroederClient]', f'[{self._id}] Generated challenge {bytes_as_hex(self._challenges[first_id])}')
        
        cipher = AES.new(session_key, AES.MODE_CTR)
        return cipher.nonce, cipher.encrypt(self._challenges[first_id])
    
    
    def _verify_response(self, nonce: bytes, response: bytes, first_id: uuid.UUID):
        cipher = AES.new(self._db[first_id], AES.MODE_CTR, nonce=nonce)
        
        response = cipher.decrypt(response)
        expected = NeedhamSchroederClient._modify_challenge(self._challenges[first_id])
        
        trace('[NeedhamSchroederClient]', f'[{self._id}] response = {bytes_as_hex(response)}')
        trace('[NeedhamSchroederClient]', f'[{self._id}] expected = {bytes_as_hex(expected)}')
        
        #
        # Если ответ совпал с ожидаемым, то все чудесно
        #
        
        result = expected == response
        if result:
            trace('[NeedhamSchroederClient]', f'[{self._id}] Established key: {bytes_as_hex(self._db[first_id])}')
            
        return result
    
    
    @staticmethod
    def _modify_challenge(challenge: bytes):
        num = number.bytes_to_long(challenge)
        return number.long_to_bytes(num - 1)

---

In [8]:
# Создаю доверенный центр
ca = NeedhamSchroederCA()

In [9]:
# Два хороших пользователя
alice = NeedhamSchroederClient()
bob   = NeedhamSchroederClient()

[TRACE] [NeedhamSchroederClient] Client 72256b44-8b45-4bbf-a8a7-c2e3ac16adff created
[TRACE] [NeedhamSchroederClient] Client 92b37b12-0fec-4b40-a393-a9f9cbf54b26 created


In [10]:
# Регистрация валидных пользователей
alice.register(ca)
bob.register(ca)

[TRACE] [NeedhamSchroederCA] Attempting to register client 72256b44-8b45-4bbf-a8a7-c2e3ac16adff 
        with key abe9cc813cc6b91220d16e94b97c5e768c86827513100481d451ffb506b9fa59
[TRACE] [NeedhamSchroederCA] Client registered successfully
[TRACE] [NeedhamSchroederCA] Attempting to register client 92b37b12-0fec-4b40-a393-a9f9cbf54b26 
        with key db53665c4ad444acea5832b76be6fd6e6c4e926cc4f52d8becdea71b199ad323
[TRACE] [NeedhamSchroederCA] Client registered successfully


In [11]:
# Протокол чудесно работает для валидных пользователей
alice.initiate_protocol(bob, ca)

[TRACE] [NeedhamSchroederClient] [72256b44-8b45-4bbf-a8a7-c2e3ac16adff] N = 1cac0fe6
[TRACE] [NeedhamSchroederCA] Accepted initiation from 72256b44-8b45-4bbf-a8a7-c2e3ac16adff to 92b37b12-0fec-4b40-a393-a9f9cbf54b26, Na = 1cac0fe6
[TRACE] [NeedhamSchroederClient] [92b37b12-0fec-4b40-a393-a9f9cbf54b26] Generated challenge 81547186
[TRACE] [NeedhamSchroederClient] [92b37b12-0fec-4b40-a393-a9f9cbf54b26] response = 81547185
[TRACE] [NeedhamSchroederClient] [92b37b12-0fec-4b40-a393-a9f9cbf54b26] expected = 81547185
[TRACE] [NeedhamSchroederClient] [92b37b12-0fec-4b40-a393-a9f9cbf54b26] Established key: fea2929ddca2fb923e9b490c5e12f24215f5a34c72765c2f30204aea840e1bec
[TRACE] [NeedhamSchroederClient] [72256b44-8b45-4bbf-a8a7-c2e3ac16adff] Established key: fea2929ddca2fb923e9b490c5e12f24215f5a34c72765c2f30204aea840e1bec


True

In [12]:
# А это злоумышленник
mallory = NeedhamSchroederClient(bob.identifier)

[TRACE] [NeedhamSchroederClient] Client 92b37b12-0fec-4b40-a393-a9f9cbf54b26 created


In [13]:
# Он не может получить ключ, если влезет в общение вместо Боба
alice.initiate_protocol(mallory, ca)

[TRACE] [NeedhamSchroederClient] [72256b44-8b45-4bbf-a8a7-c2e3ac16adff] N = 39a54fdc
[TRACE] [NeedhamSchroederCA] Accepted initiation from 72256b44-8b45-4bbf-a8a7-c2e3ac16adff to 92b37b12-0fec-4b40-a393-a9f9cbf54b26, Na = 39a54fdc
[TRACE] [NeedhamSchroederClient] [92b37b12-0fec-4b40-a393-a9f9cbf54b26] Wrong identifier
[TRACE] [NeedhamSchroederClient] [72256b44-8b45-4bbf-a8a7-c2e3ac16adff] Second client denied packet


False

In [14]:
# Также он не сможет получить ключ, если сам инициирует протокол с Алисой
mallory.initiate_protocol(alice, ca)

[TRACE] [NeedhamSchroederClient] [92b37b12-0fec-4b40-a393-a9f9cbf54b26] N = 2ff27490
[TRACE] [NeedhamSchroederCA] Accepted initiation from 92b37b12-0fec-4b40-a393-a9f9cbf54b26 to 72256b44-8b45-4bbf-a8a7-c2e3ac16adff, Na = 2ff27490
[TRACE] [NeedhamSchroederClient] [92b37b12-0fec-4b40-a393-a9f9cbf54b26] Malformed response


False